# TODOs
* clean up notebook
* parameterize
* offer large and small options for producing dataset (create optionals)

## Load env config

In [1]:
# naming convention for all cloud resources
VERSION        = "v1"                  # TODO
PREFIX         = f'ndr-{VERSION}'      # TODO

print(f"PREFIX = {PREFIX}")

PREFIX = ndr-v1


In [2]:
# staging GCS
GCP_PROJECTS             = !gcloud config get-value project
PROJECT_ID               = GCP_PROJECTS[0]

# GCS bucket and paths
BUCKET_NAME              = f'{PREFIX}-{PROJECT_ID}-bucket'
BUCKET_URI               = f'gs://{BUCKET_NAME}'

config = !gsutil cat {BUCKET_URI}/config/notebook_env.py
print(config.n)
exec(config.n)


PROJECT_ID               = "hybrid-vertex"
PROJECT_NUM              = "934903580331"
LOCATION                 = "us-central1"

REGION                   = "us-central1"
BQ_LOCATION              = "US"
VPC_NETWORK_NAME         = "ucaip-haystack-vpc-network"

VERTEX_SA                = "934903580331-compute@developer.gserviceaccount.com"

PREFIX                   = "ndr-v1"
VERSION                  = "v1"

APP                      = "sp"
MODEL_TYPE               = "2tower"
FRAMEWORK                = "tfrs"
DATA_VERSION             = "v1"
TRACK_HISTORY            = "5"

BUCKET_NAME              = "ndr-v1-hybrid-vertex-bucket"
BUCKET_URI               = "gs://ndr-v1-hybrid-vertex-bucket"
SOURCE_BUCKET            = "spotify-million-playlist-dataset"

DATA_GCS_PREFIX          = "data"
DATA_PATH                = "gs://ndr-v1-hybrid-vertex-bucket/data"
VOCAB_SUBDIR             = "vocabs"
VOCAB_FILENAME           = "vocab_dict.pkl"

CANDIDATE_PREFIX         = "candidates"
TRAIN_DIR_PREFIX      

#### Step 0: Dependencies

Run this one time when starting, then restart the kernel

In [1]:
# !pip install pandas pandas-gbq==0.12.0 --user

# Data prep

## In this notebook we will load the songs from the zip file, and perform transformations to prepare the data for two-tower training
Steps
1. Extract from the zip file
2. Upload to BQ
3. Enrich features for the playlist songs
4. Cross-join songs with features (expected rows = n_songs x n_playlists)
5. Remove after-the-fact (later position songs) from the newly generated samples
6. Create a clean train table, and flatten structs or use arrays

#### Unzip the file and upload to BQ
Source of data if you want to download zip: gs://spotify-million-playlist-dataset

In [3]:
# # Set your variables for your project, region, and dataset name
# SOURCE_BUCKET = 'spotify-million-playlist-dataset'
# PROJECT_ID = 'hybrid-vertex'
# REGION = 'us-central1'
# BQ_DATASET = 'spotify_e2e_test'

import time
from google.cloud import bigquery

bigquery_client = bigquery.Client(project=PROJECT_ID, location=BQ_LOCATION)

### Specify BQ Dataset created in `00-env-setup.ipynb`

In [4]:
# # Create a bigquery dataset (one time operation)
# # Construct a full Dataset object to send to the API.
dataset = bigquery.Dataset(f"`{PROJECT_ID}.{BQ_DATASET}`")
dataset

Dataset(DatasetReference('`hybrid-vertex', 'spotify_e2e_test`'))

## Next create unique artist and song tables
These tables contain features obtained via the public Spotify API. Features such as track and artist popularity are in this data. For more detail on loading json data to Bigquery, [see here](https://cloud.google.com/bigquery/docs/loading-data-cloud-storage-json)

![](img/unique-songs.png)

### Unique artists

![](img/unique-artists.png)

##### The data is now in BQ

## The tables are set for feature enrichment
We will visit these tables later, now let's load the Million Playlist dataset locally and push bq using `pandas-gbq` (see requirements installation at the top)

In [3]:
# !gsutil cp gs://{SOURCE_BUCKET}spotify_million_playlist_dataset.zip .
# !unzip spotify_million_playlist_dataset.zip

#### This step can take up to 30 minutes

In [ ]:
%%time
import os
import json
import pandas as pd
data_files = os.listdir('data')

#make sure there is not already existing data in the playlists table
#loops over json files - converts to pandas then upload/appends
for filename in data_files:
    with open(f'data/{filename}') as f:
        json_dict = json.load(f)
        df = pd.DataFrame(json_dict['playlists'])
        df.to_gbq(
        destination_table=f'{BQ_DATASET}.playlists', 
        project_id=PROJECT_ID, # TODO: param
        location=REGION, 
        progress_bar=False, 
        reauth=True, 
        if_exists='append'
        ) 

Now the data is loaded but the playlists are nested as one large string that needs to be parsed - we will use json compatible functionality with BigQuery to address

![](img/tracks-string.png)

### Import bigquery and run parameterized queries to shape the data

This query formats the json strings to be read as Bigquery structs, to be manipulated in subsequent queries

In [ ]:
JSON_EXTRACT_QUERY = f"""
  CREATE OR REPLACE TABLE `{PROJECT_ID}.{BQ_DATASET}.playlists_nested` as (
    WITH json_parsed as (
      SELECT * except(tracks), JSON_EXTRACT_ARRAY(tracks) as json_data 
    FROM `{PROJECT_ID}.{BQ_DATASET}.playlists` 
    )

    SELECT json_parsed.* except(json_data),
      ARRAY(
         SELECT AS STRUCT
          JSON_EXTRACT_SCALAR(json_data, "$.pos") as pos, 
          JSON_EXTRACT_SCALAR(json_data, "$.artist_name") as artist_name,
          JSON_EXTRACT_SCALAR(json_data, "$.track_uri") as track_uri,
          JSON_EXTRACT_SCALAR(json_data, "$.artist_uri") as artist_uri,
          JSON_EXTRACT_SCALAR(json_data, "$.track_name") as track_name,
          JSON_EXTRACT_SCALAR(json_data, "$.album_uri") as album_uri,
          JSON_EXTRACT_SCALAR(json_data, "$.duration_ms") as duration_ms,
          JSON_EXTRACT_SCALAR(json_data, "$.album_name") as album_name
        FROM json_parsed.json_data
      ) as tracks,
    FROM json_parsed
    ) 
"""

# print(JSON_EXTRACT_QUERY)

In [ ]:
%%time
bigquery_client.query(JSON_EXTRACT_QUERY).result()

Now `playlists_nested` has parsed the string data to a struct with arrays that will allow us to process the data much more easily

![](img/playlists-nested.png)

## Next we get the unique track features to put in a BQ table

This table will then be used to call the Spotify API and enrich with additional data about each track and artist

In [ ]:
UNIQUE_TRACKS_QUERY = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{BQ_DATASET}.tracks_unique` as (
  SELECT distinct 
    track.track_uri,
    track.album_uri,
    track.artist_uri, 
  FROM `{PROJECT_ID}.{BQ_DATASET}.playlists_nested`, UNNEST(tracks) as track
)
"""

# print(UNIQUE_TRACKS_QUERY)

CPU times: user 12.7 ms, sys: 757 µs, total: 13.4 ms
Wall time: 10.7 s


In [ ]:
%%time
bigquery_client.query(UNIQUE_TRACKS_QUERY).result()

## Now enrich the playlist songs with the new features

> Note: these tables are not included in the Spotify MPD. See `00-load-core-data-to-bq.ipynb` 

**New tables**
* `audio_features` - created from prior notebook via Spotify API
* `artist_features` - created from prior notebook via Spotify API

In [25]:
ENRICH_QUERY = f"""
CREATE OR REPLACE TABLE
  `{PROJECT_ID}.{BQ_DATASET}.enriched_data` AS (
  WITH tf as (SELECT distinct * from `{PROJECT_ID}.{BQ_DATASET}.audio_features`),
       af as (SELECT distinct * from `{PROJECT_ID}.{BQ_DATASET}.artist_features`) 

    SELECT
    a.* except(tracks),
      ARRAY(
    SELECT
      AS STRUCT CAST(track.pos AS int64) AS pos_can,
      case when track.artist_name = '' then 'NONE' else track.artist_name end AS artist_name_can,
      case when track.track_uri = '' then 'NONE' else track.track_uri  end AS track_uri_can,
      case when track.album_uri = '' then 'NONE' else track.album_uri  end AS album_uri_can,
      case when track.artist_uri = '' then 'NONE' else track.artist_uri  end AS artist_uri_can,
      case when track.track_name = '' then 'NONE' else track.track_name end AS track_name_can,
      CAST(track.duration_ms AS float64) / 1.0 AS duration_ms_can,
      case when track.album_name = '' then 'NONE' else track.album_name end AS album_name_can,
      CAST(IFNULL(tf.track_pop, 0.0) as float64) / 1.0 AS track_pop_can,
      CAST(IFNULL(af.artist_pop, 0.0) as float64) / 1.0  AS artist_pop_can,
      case when 
        ARRAY(SELECT * FROM UNNEST(SPLIT(SUBSTR(genres, 2 , LENGTH(genres) - 2))))[OFFSET(0)] = '' 
      then 
        ['NONE'] else ARRAY(SELECT * FROM UNNEST(SPLIT(SUBSTR(genres, 2 , LENGTH(genres) - 2)))) end AS artist_genres_can,
      CAST(IFNULL(af.followers, 0.0) as float64) / 1.0 AS artist_followers_can
    FROM
      UNNEST(tracks) as track
    INNER JOIN
     tf --track features
    ON
      (track.track_uri = tf.track_uri)
    INNER JOIN
      af
      ON
      (track.artist_uri = af.artist_uri)
      ) AS tracks
  FROM 
  `{PROJECT_ID}.{BQ_DATASET}.playlists_nested` as a
  )
"""

# print(ENRICH_QUERY)

CPU times: user 22.1 ms, sys: 0 ns, total: 22.1 ms
Wall time: 33.8 s


In [ ]:
%%time
bigquery_client.query(ENRICH_QUERY).result()

## Cross join + get rid of after-the-fact `pos` data in playlist

cross_join_songxplaylist_struct_query

`hybrid-vertex.spotify_train_3.ordered_position_training`

We create a data structure that creates unique song-playlist combos (every possible via cross-join). There is also a portion of pulling the last song in the playlist as the "seed track"
________
### Note on the approach

Semantic matching requires pairs, triplets (tuples generally) of co-occurrences between pairs. This is a very broad definition, and with this newer approach many new use cases are being explored. A simple example are finding pairs of user queries and purchases. The training example pair are: (the features we know from the user query, the features we know on the product they ultimately purchased).

There are other approaches where triples are considered, and there are advanced techniques on negative sampling, finding “bad” examples of query, product pairs, which we will not cover here.

Note there are other sampling techniques we highlight below (different artist/album)

The chosen task was predicting the next song on a playlist, given the playlist existing order. The approach taken was to create pairs for all children songs and their parent playlists. We did leveraging BigQuery’s `UNNEST` and `CROSS JOIN`. 

We also had rich features for playlists, albums and songs in another table that was later used to enrich post `CROSS JOIN`. This was done to optimize the computation since the cross-joining is expensive and it was subsequently much quicker to enrich after this step.

Now that we completed this step, we had all combinations of child song, playlist pairs. The song was the candidate label but the playlist still contained the candidate label and all songs after. Additional criteria was added to remove the candidate song and all songs that occur after the candidate in the playlist. For the sake of performance we also only considered the last 5 played songs. Other sampling configurations are available in the example notebook as well (only predicting when there are album and artist switches).

What this results in is a training dataset that has all possible child song candidates joined with the full playlist data, and the playlist data is properly censored as to only contain songs up to before the candidate song.

![](img/semantic-pair.png)

In [30]:
CROSS_JOIN_QUERY = f"""
  CREATE OR REPLACE TABLE
  `{PROJECT_ID}.{BQ_DATASET}.ordered_position_training` AS (
  WITH
    -- get every combination of song and its parent playlist
    unnest_cross AS(
    SELECT
      b.*,
      CONCAT(b.pid,"-",track.pos_can) AS pid_pos_id,
      CAST(track.pos_can AS int64) AS pos_can,
      IFNULL(track.artist_name_can, "NONE") as artist_name_can ,
      track.track_uri_can ,
      track.album_uri_can,
      IFNULL(track.track_name_can, "NONE") as track_name_can ,
      track.artist_uri_can ,
      CAST(track.duration_ms_can AS float64) AS duration_ms_can,
      track.album_name_can ,
      track.track_pop_can ,
      track.artist_pop_can,
      ARRAY_TO_STRING(track.artist_genres_can, ',', 'MISSING') as artist_genres_can ,
      track.artist_followers_can 
    FROM (
      SELECT
        * EXCEPT(duration_ms)
      FROM
        `{PROJECT_ID}.{BQ_DATASET}.enriched_data`) AS b
    CROSS JOIN
      UNNEST(tracks) AS track)
  SELECT
    a.* EXCEPT(tracks,
      num_tracks,
      num_artists,
      num_albums,
      num_followers,
      num_edits),
    ARRAY(
    SELECT
      AS STRUCT CAST(track.pos_can AS int64) AS pos_pl,
      track.artist_name_can AS artist_name_pl,
      track.track_uri_can AS track_uri_pl,
      track.track_name_can AS track_name_pl,
      track.album_uri_can AS album_uri_pl,
      track.artist_uri_can AS artist_uri_pl,
      CAST(track.duration_ms_can AS float64) AS duration_ms_pl,
      track.album_name_can AS album_name_pl,
      track.track_pop_can AS track_pop_pl,
      track.artist_pop_can AS artist_pop_pl,
      ARRAY_TO_STRING(track.artist_genres_can, ',', 'MISSING') AS artist_genres_pl,
      track.artist_followers_can AS artist_followers_pl,
    FROM
      UNNEST(tracks) AS track
    WHERE
      CAST(track.pos_can AS int64) < a.pos_can ORDER BY CAST(track.pos_can AS int64)) AS seed_playlist_tracks
  FROM
    unnest_cross AS a -- with statement
  )
"""

# print(CROSS_JOIN_QUERY)

CPU times: user 37.7 ms, sys: 94 µs, total: 37.8 ms
Wall time: 1min 46s


In [ ]:
%%time
bigquery_client.query(CROSS_JOIN_QUERY).result()

## Update the playlist metadata with the new samples created above

Add audio features from the tracks

Get new metadata for the tracks now that there are updated track counts, durations, etc...

`hybrid-vertex.spotify_train_3.train` will be produced


In [48]:
GET_NEW_METADATA_QUERY = f"""
    CREATE OR REPLACE TABLE `{PROJECT_ID}.{BQ_DATASET}.train` as (
    WITH
      playlist_features_clean AS (
      SELECT
        pid_pos_id,
        SUM(trx.duration_ms_pl) / 1.0 AS duration_ms_seed_pl,
        COUNT(1) / 1.0 AS n_songs_pl,
        COUNT(DISTINCT trx.artist_name_pl) / 1.0 AS num_artists_pl,
        COUNT(DISTINCT trx.album_uri_pl) /1.0 AS num_albums_pl,
      FROM
        `{PROJECT_ID}.{BQ_DATASET}.ordered_position_training`,
        UNNEST(seed_playlist_tracks) AS trx
      GROUP BY
        pid_pos_id
        )
    SELECT
      a.* except(artist_genres_can, track_pop_can, artist_pop_can, artist_followers_can),
      b.* except(pid_pos_id),
      a.artist_genres_can,
      IFNULL(a.track_pop_can, 0.0) / 1.0 as  track_pop_can, 
      IFNULL(a.artist_pop_can, 0.0) / 1.0 as artist_pop_can,
      IFNULL(a.artist_followers_can, 0.0) / 1.0 as artist_followers_can,

    FROM
      `{PROJECT_ID}.{BQ_DATASET}.ordered_position_training` a
    INNER JOIN
      playlist_features_clean b
    ON
      a.pid_pos_id = b.pid_pos_id 
    )
"""

# print(GET_NEW_METADATA_QUERY)

CPU times: user 38.8 ms, sys: 607 µs, total: 39.4 ms
Wall time: 1min 54s


In [ ]:
%%time
bigquery_client.query(GET_NEW_METADATA_QUERY).result()

In [133]:
### Get candidates

GET_UNIQUE_CANDIDATES_QUERY = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{BQ_DATASET}.{BQ_TABLE_CANDIDATES}` AS (
    WITH
      af as (SELECT DISTINCT * FROM `{PROJECT_ID}.{BQ_DATASET}.audio_features`)

    SELECT DISTINCT
        track_uri_can,
        track_name_can,
        artist_uri_can,
        artist_name_can,
        album_uri_can,
        album_name_can,
        duration_ms_can,
        track_pop_can,
        artist_pop_can,
        artist_genres_can,
        artist_followers_can,

        IFNULL(af.danceability, 0.) as track_danceability_can,
        IFNULL(af.energy, 0.) as track_energy_can,
        IFNULL(af.key, 0.) as track_key_can,
        IFNULL(af.loudness, 0.) as track_loudness_can,
        IFNULL(af.mode, 0) as track_mode_can,
        IFNULL(af.speechiness, 0.) as track_speechiness_can,
        IFNULL(af.acousticness, 0.) as track_acousticness_can,
        IFNULL(af.instrumentalness, 0.) as track_instrumentalness_can,
        IFNULL(af.liveness, 0.) as track_liveness_can,
        IFNULL(af.valence, 0.) as track_valence_can,
        IFNULL(af.tempo, 0.) as track_tempo_can,
        IFNULL(af.time_signature, 0) as time_signature_can,
    FROM
      `{PROJECT_ID}.{BQ_DATASET}.train` a
       inner join af on af.track_uri = a.track_uri_can
       
    )
"""

# print(GET_UNIQUE_CANDIDATES_QUERY)

CPU times: user 52.2 ms, sys: 161 µs, total: 52.4 ms
Wall time: 15.7 s


In [ ]:
%%time
bigquery_client.query(GET_UNIQUE_CANDIDATES_QUERY).result()

## For TFRecords
Get rid of structs by creating new table with arrays from playlist_seed


# Only selecting last 5 songs

song_history is settable but it will impact `MAX_PLAYLIST_LENGTH` in `src/two_tower.py`

In [7]:
TRAIN_FLATTEN_QUERY_a = f"""
CREATE OR REPLACE TABLE
  `{PROJECT_ID}.{BQ_DATASET}.train_flatten_pre_split_a` AS (
  WITH
    audio AS (
    SELECT
      DISTINCT *
    FROM
      `{PROJECT_ID}.{BQ_DATASET}.audio_features`)
  SELECT
    pid,
    IFNULL(a.name, "") AS pl_name_src,
    collaborative AS pl_collaborative_src,
    duration_ms_seed_pl AS pl_duration_ms_new,
    n_songs_pl AS num_pl_songs_new,
    num_artists_pl AS num_pl_artists_new,
    num_albums_pl AS num_pl_albums_new,
    track_uri_can,
    track_name_can,
    artist_uri_can,
    artist_name_can,
    album_uri_can,
    album_name_can,
    duration_ms_can,
    track_pop_can,
    artist_pop_can,
    artist_genres_can,
    artist_followers_can,
    IFNULL(audio.danceability, 0.0) AS track_danceability_can,
    IFNULL(audio.energy, 0.0) AS track_energy_can,
    IFNULL(audio.key, 0.0) AS track_key_can,
    IFNULL(audio.loudness, 0.0) AS track_loudness_can,
    IFNULL(audio.mode, 0) AS track_mode_can,
    IFNULL(audio.acousticness, 0.0) AS track_acousticness_can,
    IFNULL(audio.instrumentalness, 0.0) AS track_instrumentalness_can,
    IFNULL(audio.liveness, 0.0) AS track_liveness_can,
    IFNULL(audio.speechiness, 0.0) AS track_speechiness_can,
    IFNULL(audio.valence, 0.0) AS track_valence_can,
    IFNULL(audio.tempo, 0.0) AS track_tempo_can,
    IFNULL(audio.time_signature, 0) AS track_time_signature_can,
    ARRAY(
      SELECT CAST(pos_can - pos_pl) AS FLOAT64
    FROM
      UNNEST(seed_playlist_tracks) t
    WHERE 
      pos_pl >= pos_can - {TRACK_HISTORY}) AS candidate_rank,
    ARRAY(
    SELECT
      t.artist_name_pl
    FROM
      UNNEST(seed_playlist_tracks) t
    WHERE
      pos_pl >= pos_can - {TRACK_HISTORY}) AS artist_name_pl,
    ARRAY(
    SELECT
      t.artist_uri_pl
    FROM
      UNNEST(seed_playlist_tracks) t
    WHERE
      pos_pl >= pos_can - {TRACK_HISTORY}) AS artist_uri_pl,
    ARRAY(
    SELECT
      t.track_uri_pl
    FROM
      UNNEST(seed_playlist_tracks) t
    WHERE
      pos_pl >= pos_can - {TRACK_HISTORY}) AS track_uri_pl,
    ARRAY(
    SELECT
      t.track_name_pl
    FROM
      UNNEST(seed_playlist_tracks) t
    WHERE
      pos_pl >= pos_can - {TRACK_HISTORY}) AS track_name_pl,
    ARRAY(
    SELECT
      t.duration_ms_pl
    FROM
      UNNEST(seed_playlist_tracks) t
    WHERE
      pos_pl >= pos_can - {TRACK_HISTORY}) AS duration_ms_songs_pl,
    ARRAY(
    SELECT
      t.album_name_pl
    FROM
      UNNEST(seed_playlist_tracks) t
    WHERE
      pos_pl >= pos_can - {TRACK_HISTORY}) AS album_name_pl,
    ARRAY(
    SELECT
      t.album_uri_pl
    FROM
      UNNEST(seed_playlist_tracks) t
    WHERE
      pos_pl >= pos_can - {TRACK_HISTORY}) AS album_uri_pl,
    ARRAY(
    SELECT
      CAST(t.artist_pop_pl AS FLOAT64)
    FROM
      UNNEST(seed_playlist_tracks) t
    WHERE
      pos_pl >= pos_can - {TRACK_HISTORY}) AS artist_pop_pl,
    ARRAY(
    SELECT
      t.artist_followers_pl
    FROM
      UNNEST(seed_playlist_tracks) t
    WHERE
      pos_pl >= pos_can - {TRACK_HISTORY}) AS artists_followers_pl,
    ARRAY(
    SELECT
      t.track_pop_pl
    FROM
      UNNEST(seed_playlist_tracks) t
    WHERE
      pos_pl >= pos_can - {TRACK_HISTORY}) AS track_pop_pl,
    ARRAY(
    SELECT
      t.artist_genres_pl
    FROM
      UNNEST(seed_playlist_tracks) t
    WHERE
      pos_pl >= pos_can - {TRACK_HISTORY}) AS artist_genres_pl
  FROM
    `{PROJECT_ID}.{BQ_DATASET}.train` a
  INNER JOIN
    audio
  ON
    audio.track_uri = a.track_uri_can 
)
"""

print(TRAIN_FLATTEN_QUERY_a)

In [8]:
%%time
bigquery_client.query(TRAIN_FLATTEN_QUERY_a).result()

CPU times: user 65 ms, sys: 6.87 ms, total: 71.9 ms
Wall time: 58 s


#### Append the audio features

In [10]:
TRAIN_FLATTEN_QUERY_b = f"""
CREATE OR REPLACE TABLE 
`{PROJECT_ID}.{BQ_DATASET}.train_flatten_pre_split_b` as (
WITH 
  audio as (SELECT DISTINCT * FROM `{PROJECT_ID}.{BQ_DATASET}.audio_features`)
SELECT 
    a.*,
    ARRAY(select IFNULL(audio.danceability, 0.0) from UNNEST(a.track_uri_pl) t, audio where audio.track_uri = t) as track_danceability_pl,
    ARRAY(select IFNULL(audio.energy, 0.0) from UNNEST(a.track_uri_pl) t, audio where audio.track_uri = t) as track_energy_pl,
    ARRAY(select IFNULL(audio.key, 0.0) from UNNEST(a.track_uri_pl) t, audio where audio.track_uri = t) as track_key_pl,
    ARRAY(select IFNULL(audio.loudness, 0.0) from UNNEST(a.track_uri_pl) t, audio where audio.track_uri = t) as track_loudness_pl,
    ARRAY(select IFNULL(audio.mode, 0) from UNNEST(a.track_uri_pl) t, audio where audio.track_uri = t) as track_mode_pl,
    ARRAY(select IFNULL(audio.acousticness, 0.0) from UNNEST(a.track_uri_pl) t, audio where audio.track_uri = t) as track_acousticness_pl,
    ARRAY(select IFNULL(audio.instrumentalness, 0.0) from UNNEST(a.track_uri_pl) t, audio where audio.track_uri = t) as track_instrumentalness_pl,
    ARRAY(select IFNULL(audio.liveness, 0.0) from UNNEST(a.track_uri_pl) t, audio where audio.track_uri = t) as track_liveness_pl,
    ARRAY(select IFNULL(audio.valence, 0.0) from UNNEST(a.track_uri_pl) t, audio where audio.track_uri = t) as track_valence_pl,
    ARRAY(select IFNULL(audio.tempo, 0.0) from UNNEST(a.track_uri_pl) t, audio where audio.track_uri = t) as track_tempo_pl,
    ARRAY(select IFNULL(audio.time_signature, 0) from UNNEST(a.track_uri_pl) t, audio where audio.track_uri = t) as track_time_signature_pl,
    ARRAY(select IFNULL(audio.speechiness, 0) from UNNEST(a.track_uri_pl) t, audio where audio.track_uri = t) as track_speechiness_pl
    from `{PROJECT_ID}.{BQ_DATASET}.train_flatten_pre_split_a` a
  WHERE
     ARRAY_LENGTH(a.track_uri_pl) = {TRACK_HISTORY}
) --limiting here for performance
"""

# print(TRAIN_FLATTEN_QUERY_b)

In [ ]:
%%time
bigquery_client.query(TRAIN_FLATTEN_QUERY_b).result()

## Important for validation strategy
Different playlist ids were selected for validation to prevent cross-contamination with the sampling approach.

In [12]:
# VALIDATION_P = 0.01

In [13]:
VALIDATION_DATA_QUERY = f"""
CREATE OR REPLACE TABLE
  `{PROJECT_ID}.{BQ_DATASET}.{BQ_TABLE_VALID}` AS (
    SELECT * 
  FROM
    `{PROJECT_ID}.{BQ_DATASET}.train_flatten_pre_split_b` where MOD(pid, 100) = 0
    AND ARRAY_LENGTH(track_uri_pl) = {TRACK_HISTORY})""" #complete examples only

# print(VALIDATION_DATA_QUERY)

In [14]:
%%time
bigquery_client.query(VALIDATION_DATA_QUERY).result()

CPU times: user 16.4 ms, sys: 2.24 ms, total: 18.6 ms
Wall time: 26.2 s


In [15]:
TRAIN_DATA_QUERY = f"""
CREATE OR REPLACE TABLE
  `{PROJECT_ID}.{BQ_DATASET}.{BQ_TABLE_TRAIN}` AS (
    SELECT * 
  FROM
    `{PROJECT_ID}.{BQ_DATASET}.train_flatten_pre_split_b` where MOD(pid, 100) != 0
    AND ARRAY_LENGTH(track_uri_pl) = {TRACK_HISTORY})"""

# print(TRAIN_DATA_QUERY)

In [16]:
%%time
bigquery_client.query(TRAIN_DATA_QUERY).result()

CPU times: user 34.1 ms, sys: 977 µs, total: 35.1 ms
Wall time: 1min 29s


## Done - you can move on to the [next notebook](02-tfrecord-beam-pipeline.ipynb) 

Your data should look like this:
    
![](img/train-dataset-metadata.png)

**Finished**